# Nashville Housing Market Analysis

This notebook explores a small sample of **Nashville housing data** to demonstrate a typical data analytics workflow:

- Load and inspect raw data  
- Clean and transform the dataset  
- Engineer new features (e.g., price per square foot, house age)  
- Analyze price patterns by neighborhood  
- Export a cleaned dataset for visualization in Tableau

The raw data is stored in: `data/nashville_housing_raw.csv`.


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Display plots inline if running in Jupyter
%matplotlib inline

## 1. Load the Dataset

In [ ]:
# Load the raw Nashville housing data
file_path = "data/nashville_housing_raw.csv"
df = pd.read_csv(file_path)

# Preview the first few rows
df.head()

## 2. Quick Overview of the Data

In [ ]:
# Basic info about columns and data types
df.info()

In [ ]:
# Summary statistics for numeric columns
df.describe(include='all')

## 3. Data Cleaning

In [ ]:
# Drop any exact duplicate rows
df = df.drop_duplicates()

# Ensure numeric columns are numeric (in case they were read as strings)
numeric_cols = ["sale_price", "bedrooms", "bathrooms", "square_feet", "year_built", "renovated_year"]
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors="coerce")

# Handle missing values in renovated_year by setting them to 0 (meaning 'never renovated')
df["renovated_year"] = df["renovated_year"].fillna(0)

# Check for remaining missing values
df.isna().sum()

## 4. Feature Engineering

In [ ]:
# Create price per square foot
df["price_per_sqft"] = df["sale_price"] / df["square_feet"]

# Create house age (assuming current year 2025)
CURRENT_YEAR = 2025
df["house_age"] = CURRENT_YEAR - df["year_built"]

# Create renovation age (0 if never renovated)
def compute_reno_age(row):
    if row["renovated_year"] == 0:
        return 0
    return CURRENT_YEAR - row["renovated_year"]

df["renovation_age"] = df.apply(compute_reno_age, axis=1)

df.head()

## 5. Neighborhood-Level Summary

In [ ]:
# Group by neighborhood to compute summary statistics
neighborhood_stats = (
    df.groupby("neighborhood")
      .agg(
          avg_price=("sale_price", "mean"),
          avg_price_per_sqft=("price_per_sqft", "mean"),
          avg_house_age=("house_age", "mean"),
          count_sales=("id", "count")
      )
      .reset_index()
)

neighborhood_stats.sort_values("avg_price", ascending=False)

## 6. Visualization: Average Price by Neighborhood

In [ ]:
# Plot the top neighborhoods by average sale price
top_neighborhoods = neighborhood_stats.sort_values("avg_price", ascending=False)

plt.figure(figsize=(10, 5))
plt.bar(top_neighborhoods["neighborhood"], top_neighborhoods["avg_price"])
plt.xticks(rotation=45, ha="right")
plt.ylabel("Average Sale Price ($)")
plt.title("Average Home Sale Price by Neighborhood (Sample Data)")
plt.tight_layout()
plt.show()

## 7. Export Cleaned Data for Tableau

In [ ]:
# Export the cleaned dataset for use in Tableau
output_path = "data/nashville_housing_cleaned.csv"
df.to_csv(output_path, index=False)

output_path

## 8. Key Takeaways (Example Talking Points)

When using this project in interviews, you can highlight points like:

- How you **cleaned and validated** the data (duplicates, data types, missing values).  
- How you **engineered features** such as price per square foot and house age.  
- How you **compared neighborhoods** using grouped statistics and visualizations.  
- How you **exported a clean dataset for Tableau** to build an interactive dashboard.

You can extend this notebook by:

- Adding more records to the dataset  
- Incorporating additional features (lot size, garage, school ratings, etc.)  
- Building advanced visualizations and trend analysis in Tableau.
